### Importing the Necessary Libraries

In [11]:
import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import random
import pickle
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [36]:
text_df = pd.read_csv('fake_or_real_news.csv')

In [37]:
text_df.head()

,id,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [38]:
text = list(text_df.text.values)
joined_text = " ".join(text)

In [73]:
partial_text = joined_text[:300000]

In [74]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text.lower())

In [75]:
unique_tokens = np.unique(tokens)
unique_token_index = {token:idx for idx, token in enumerate(unique_tokens)}

In [76]:
n_words = 10
input_words =  []
next_words = []

for i in range(len(tokens) - n_words):
  input_words.append(tokens[i:i + n_words])
  next_words.append(tokens[i + n_words])

In [77]:
X = np.zeros((len(input_words),n_words,len(unique_tokens)),dtype='bool')
y = np.zeros((len(next_words),len(unique_tokens)),dtype='bool')

In [78]:
for i, words in enumerate(input_words):
  for j, word in enumerate(words):
    X[i, j, unique_token_index[word]] = 1
    y[i, unique_token_index[next_words[i]]] = 1

In [79]:
model = Sequential()
model.add(LSTM(128, input_shape=(n_words,len(unique_tokens)),return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [80]:
model.compile(loss ="categorical_crossentropy",optimizer=RMSprop(learning_rate=0.01),metrics=["accuracy"])
model.fit(X,y,batch_size=128,epochs=50,shuffle=True)

Epoch 1/50
400/400 [==============================] - 50s 119ms/step - loss: 7.1140 - accuracy: 0.0551
Epoch 2/50
400/400 [==============================] - 45s 113ms/step - loss: 6.9091 - accuracy: 0.0581
Epoch 3/50
400/400 [==============================] - 43s 109ms/step - loss: 6.9325 - accuracy: 0.0670
Epoch 4/50
400/400 [==============================] - 42s 105ms/step - loss: 6.9174 - accuracy: 0.0774
Epoch 5/50
400/400 [==============================] - 42s 106ms/step - loss: 6.9951 - accuracy: 0.0925
Epoch 6/50
400/400 [==============================] - 42s 105ms/step - loss: 7.0118 - accuracy: 0.1009
Epoch 7/50
400/400 [==============================] - 43s 106ms/step - loss: 7.8533 - accuracy: 0.0926
Epoch 8/50
400/400 [==============================] - 42s 106ms/step - loss: 6.8601 - accuracy: 0.1168
Epoch 9/50
400/400 [==============================] - 42s 105ms/step - loss: 6.7007 - accuracy: 0.1294
Epoch 10/50
400/400 [==============================] - 41s 103ms/step - l

In [81]:
model.save("next_word_prediction.keras")

In [82]:
def predict_next_word(input_text,n_best):
  input_text = input_text.lower()
  X = np.zeros((1,n_words, len(unique_tokens)))
  for i, word in enumerate(input_text.split()):
    X[0, i, unique_token_index[word]] = 1
  predictions = model.predict(X)[0]
  return np.argpartition(predictions, -n_best)[-n_best:]

In [98]:
possible = predict_next_word("The Park is ",5)

1/1 [==============================] - 0s 15ms/step


In [99]:
print([unique_tokens[idx] for idx in possible])

['common', 'giving', 'using', 'nuclear', 'three']


# The below code is for text generation which can be included to the scope of our project

In [85]:
def generate_text(input_text, text_length,creativity=3):
  word_sequence = input_text.split()
  current = 0
  for _ in range(text_length):
    sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower()))[current: current+n_words]
    try:
      choice = unique_tokens[random.choice(predict_next_word(sub_sequence,creativity))]
    except:
      choice = random.choice(unique_tokens)
    word_sequence.append(choice)
    current+=1
  return " ".join(word_sequence)

In [97]:
generate_text("US tax payers are suffering a lot due to", 12, 3)

1/1 [==============================] - 0s 14ms/step


'US tax payers are suffering a lot due to angry machine people chancellor laconia down coup moreno misleading nightlife rendering brooklyn'